
using CUDA
n_atoms = 100
atom_mass = 10.0f0u"u"
boundary = CubicBoundary(2.0f0u"nm", 2.0f0u"nm", 2.0f0u"nm")
temp = 100.0f0u"K"
atoms = CuArray([Atom(mass=atom_mass, σ=0.3f0u"nm", ϵ=0.2f0u"kJ * mol^-1") for i in 1:n_atoms])
coords = CuArray(place_atoms(n_atoms, boundary; min_dist=0.3u"nm"))
velocities = CuArray([velocity(atom_mass, temp) for i in 1:n_atoms])
simulator = VelocityVerlet(dt=0.002f0u"ps")

sys = System(
    atoms=atoms,
    pairwise_inters=(LennardJones(),),
    coords=coords,
    velocities=velocities,
    boundary=boundary,
    loggers=(
        temp=TemperatureLogger(typeof(1.0f0u"K"), 10),
        coords=CoordinateLogger(typeof(1.0f0u"nm"), 10),
    ),
)

simulate!(sys, simulator, 1_000)
### Let us simulate a diatomic molecule
##### If we want to define specific interactions between atoms, for example bonds, we can do this as well. Using the same definitions as the first example, let's set up the coordinates so that paired atoms are 1 Å apart.
## 100 atoms
## cubic
coords = place_atoms(n_atoms ÷ 2, boundary; min_dist=0.3u"nm")
for i in 1:length(coords)
    push!(coords, coords[i] .+ [0.1, 0.0, 0.0]u"nm")
end

velocities = [velocity(atom_mass, temp) for i in 1:n_atoms]
#### Now we can use the built-in interaction list and bond types to place harmonic bonds between paired atoms.
bonds = InteractionList2Atoms(
    collect(1:(n_atoms ÷ 2)),           # First atom indices
    collect((1 + n_atoms ÷ 2):n_atoms), # Second atom indices
    repeat([""], n_atoms ÷ 2),          # Bond types
    [HarmonicBond(k=300_000.0u"kJ * mol^-1 * nm^-2", r0=0.1u"nm") for i in 1:(n_atoms ÷ 2)],
)

specific_inter_lists = (bonds,)
##### This time, we are also going to use a neighbor list to speed up the Lennard Jones calculation. We can use the built-in ```DistanceNeighborFinder```. The arguments are a 2D array of eligible interacting pairs, the number of steps between each update and the distance cutoff to be classed as a neighbor. Since the neighbor finder is run every 10 steps we should also use a cutoff for the interaction with a cutoff distance less than the neighbor list distance.
# All pairs apart from bonded pairs are eligible for non-bonded interactions
nb_matrix = trues(n_atoms, n_atoms)
for i in 1:(n_atoms ÷ 2)
    nb_matrix[i, i + (n_atoms ÷ 2)] = false
    nb_matrix[i + (n_atoms ÷ 2), i] = false
end

neighbor_finder = DistanceNeighborFinder(
    nb_matrix=nb_matrix,
    n_steps=10,
    dist_cutoff=1.5u"nm",
)

pairwise_inters = (LennardJones(nl_only=true, cutoff=DistanceCutoff(1.2u"nm")),)
sys = System(
    atoms=atoms,
    pairwise_inters=pairwise_inters,
    specific_inter_lists=specific_inter_lists,
    coords=coords,
    velocities=velocities,
    boundary=boundary,
    neighbor_finder=neighbor_finder,
    loggers=(
        temp=TemperatureLogger(10),
        coords=CoordinateLogger(10),
    ),
)

simulator = VelocityVerlet(
    dt=0.002u"ps",
    coupling=AndersenThermostat(temp, 1.0u"ps"),
)
simulate!(sys, simulator, 1_000)
#### This time when we view the trajectory we can add lines to show the bonds.
visualize(
    sys.loggers.coords,
    boundary,
    "sim_diatomic.mp4";
    connections=[(i, i + (n_atoms ÷ 2)) for i in 1:(n_atoms ÷ 2)],
)
### Simulating the **protein**

### Simulating gravity
##### Molly is geared primarily to molecular simulation, but can also be used to simulate other physical systems. Let's set up a gravitational simulation. This example also shows the use of Float32, a 2D simulation and no specified units.
atoms = [Atom(mass=1.0f0), Atom(mass=1.0f0)]
coords = [SVector(0.3f0, 0.5f0), SVector(0.7f0, 0.5f0)]
velocities = [SVector(0.0f0, 1.0f0), SVector(0.0f0, -1.0f0)]
pairwise_inters = (Gravity(nl_only=false, G=1.5f0),)
simulator = VelocityVerlet(dt=0.002f0)
boundary = RectangularBoundary(1.0f0, 1.0f0)

sys = System(
    atoms=atoms,
    pairwise_inters=pairwise_inters,
    coords=coords,
    velocities=velocities,
    boundary=boundary,
    loggers=(coords=CoordinateLogger(Float32, 10; dims=2),),
    force_units=NoUnits,
    energy_units=NoUnits,
)

simulate!(sys, simulator, 2_000)
visualize(
    sys.loggers.coords,
    boundary,
    "sim_gravity.mp4";
    trails=4,
    framerate=15,
    color=[:orange, :lightgreen],
)
### A **fun topic**: simulating the solar system
using GLMakie
using Molly

# Using get_body_barycentric_posvel from Astropy
coords = [
    SVector(-1336052.8665050615,  294465.0896030796 ,  158690.88781384667)u"km",
    SVector(-58249418.70233503 , -26940630.286818042, -8491250.752464907 )u"km",
    SVector( 58624128.321813114, -81162437.2641475  , -40287143.05760552 )u"km",
    SVector(-99397467.7302648  , -105119583.06486066, -45537506.29775053 )u"km",
    SVector( 131714235.34070954, -144249196.60814604, -69730238.5084304  )u"km",
]

velocities = [
    SVector(-303.86327859262457, -1229.6540090943934, -513.791218405548  )u"km * d^-1",
    SVector( 1012486.9596885007, -3134222.279236384 , -1779128.5093088674)u"km * d^-1",
    SVector( 2504563.6403826815,  1567163.5923297722,  546718.234192132  )u"km * d^-1",
    SVector( 1915792.9709661514, -1542400.0057833872, -668579.962254351  )u"km * d^-1",
    SVector( 1690083.43357355  ,  1393597.7855017239,  593655.0037930267 )u"km * d^-1",
]

body_masses = [
    1.989e30u"kg",
    0.330e24u"kg",
    4.87e24u"kg" ,
    5.97e24u"kg" ,
    0.642e24u"kg",
]

boundary = CubicBoundary(1e9u"km", 1e9u"km", 1e9u"km")

# Convert the gravitational constant to the appropriate units
inter = Gravity(G=convert(typeof(1.0u"km^3 * kg^-1 * d^-2"), Unitful.G))

sys = System(
    atoms=[Atom(mass=m) for m in body_masses],
    pairwise_inters=(inter,),
    coords=coords .+ (SVector(5e8, 5e8, 5e8)u"km",),
    velocities=velocities,
    boundary=boundary,
    loggers=(coords=CoordinateLogger(typeof(1.0u"km"), 10),),
    force_units=u"kg * km * d^-2",
    energy_units=u"kg * km^2 * d^-2",
)

simulator = Verlet(
    dt=0.1u"d",
    remove_CM_motion=false,
)

simulate!(sys, simulator, 3650) # 1 year

visualize(
    sys.loggers.coords,
    boundary,
    "sim_planets.mp4";
    trails=5,
    color=[:yellow, :grey, :orange, :blue, :red],
    markersize=[0.25, 0.08, 0.08, 0.08, 0.08],
    transparency=false,
)

### Julia for Machine Learning
- Recommending packages
![](FluxLogo.png)
- Showing tutorials
[Machine Learning with Julia Language](https://youtu.be/FLAjNjcwDpI)
### Building your first model
#### This is a simple linear regression model that attempts to recover a linear function by looking at noisy examples.
#### Do not forget to install Flux using ```Pkg.install("Flux")```
##### First, we’ll write a function that generates our “true” data. We’ll use to use Flux to recover W_truth and b_truth by looking only at examples of the ground_truth function.
using Flux

# Define the ground truth model. We aim to recover W_truth and b_truth using
# only examples of ground_truth()
W_truth = [1 2 3 4 5;
            5 4 3 2 1]
b_truth = [-1.0; -2.0]
ground_truth(x) = W_truth*x .+ b_truth
#### Next, we generate our training data by passing random vectors into the ground truth function. We’ll also add Gaussian noise using randn() so that it’s not too easy for Flux to figure out the model.
x_train = [ 5 .* rand(5) for _ in 1:10_000 ]
y_train = [ ground_truth(x) + 0.2 .* randn(2) for x in x_train ]
#### There are two important things to note in this example which differ from real machine learning problems:
 - ### Our variables are individual vectors, stored inside another vector. Usually, we would have a collection of N-dimensional arrays (N >= 2) as our data.
 - ### In a real learning scenario, we would not have access to our ground truth, only the training examples.
#### Next, we define the model we want to use to learn the data. We’ll use the same form that we used for our training data:
model(x) = W*x .+ b
#### We need to set the parameters of the model (W and b) to some initial values. It’s fairly common to use random values, so we’ll do that:
W = rand(2, 5)
b = rand(2)
#### A loss function evaluates a machine learning model’s performance. In other words, it measures how far the model is from its target prediction. Flux lets you define your own custom loss function, or you can use one of the Loss Functions that Flux provides.

#### For this example, we’ll define a loss function that measures the squared distance from the predicted output to the actual output:
function loss(x, y)
    ŷ = model(x)
    sum((y .- ŷ).^2)
  end
#### You train a machine learning model by running an optimization algorithm (optimiser) that finds the best parameters (W and b). The best parameters for a model are the ones that achieve the best score of the loss function. Flux provides Optimisers that you can use to train a model.

#### For this tutorial, we’ll use a classic gradient descent optimiser with learning rate η = 0.01:
opt = Descent(0.01)
#### Training a model is the process of computing the gradients with respect to the parameters for each input in the data. At every step, the optimiser updates all of the parameters until it finds a good value for them. This process can be written as a loop: we iterate over the examples in x_train and y_train and update the model for each example.

#### To indicate that we want all derivatives of W and b, we write ps = Flux.params(W, b). This is a convenience function that Flux provides so that we don’t have to explicitly list every gradient we want. Check out the section on Taking Gradients if you want to learn more about how this works.

#### We can now execute the training procedure for our model:
train_data = zip(x_train, y_train)
ps = Flux.params(W, b)

for (x,y) in train_data
  gs = Flux.gradient(ps) do
    loss(x,y)
  end
  Flux.Optimise.update!(opt, ps, gs)
end
#### Instead of writing your own loop, you can use the built-in method on Flux:
Flux.train!(loss, Flux.params(W, b), train_data, opt)
#### The training loop we ran modified W and b to be closer to the values used to generate the training data (W and b). We can see how well we did by printing out the difference between the learned and actual matrices.
@show W
@show maximum(abs, W .- W_truth)
#### Because the data and initialization are random, your results may vary slightly, but in most cases, the largest difference between the elements of learned and actual W matrix is no more than 4%.
### Conclusion: **Julia is a super powerful language**